# Calculate Run Value from Pitch by Pitch Data

## Purpose:
### Prepare model to apply to synergy data to create similar matrix to the college baseball run environment, more specifically the CAA.

## Deriving run values from pitch by pitch data from the 2022 regular season:
### 1. Create a column containing the count-out-base state for each pitch.
### 2. Split the data into individual half-innings.
### 3. For each half-inning, create a new column that holds the difference in runs scored by the end of the half-inning compared to the beginning.
### 4. Iterate through each pitch, adding the value in the 'runs_by_end' column to the existing run value for that state and increasing the count of times that state has occurred in a dictionary of states and run values. If the state is not already in the dictionary, add it with the value of 'runs_by_end' and a count of 1.
### 5. For each state in the dictionary, divide the total run value by the number of times that state occurred to calculate the average runs scored by the end of the half-inning when that state is present.
### 6. Create a new dictionary with the states as keys and the average runs scored as values.

In [1]:
import pybaseball as pyb
import pandas as pd

In [2]:
def create_half_inning_df(group):
    # Create a new dataframe for the half-inning
    half_inning_df = pd.DataFrame(group)
        
    return half_inning_df

def map_state_column(df, run_expectancy_dict):
    df['run_expectancy'] = df['state'].map(run_expectancy_dict)
    return df

def calc_delta_run_expectancy(df):
    # Shift the values in the 'run_expectancy' column down by one row
    df['run_expectancy_shifted'] = df['run_expectancy'].shift(-1).fillna(0)
    # Create a new column with the difference between the shifted 'run_expectancy' and the current 'run_expectancy'
    df['delta_run_expectancy'] = df['run_expectancy_shifted'] - df['run_expectancy']
    # For the last row in the dataframe, set the 'delta_run_expectancy' to 0 - 'run_expectancy'
    df.loc[df.index[-1], 'delta_run_expectancy'] = 0 - df['run_expectancy'].iloc[-1]
    return df

def event_rv(event):
    event_rv = pxp_updated.loc[pxp_updated['event'] == event]
    event_rv = event_rv[['state','delta_run_expectancy']].groupby(['state']).mean()
    return event_rv

In [3]:
neu = pd.read_csv('2022_neu_hitter.csv', index_col=False, usecols=[2,5,7,8,9,10,11,12,13,14,15,16,17,18])
coc = pd.read_csv('2022_coc_hitter.csv', index_col=False, usecols=[2,5,7,8,9,10,11,12,13,14,15,16,17,18])
dele = pd.read_csv('2022_deleware_hitters.csv', index_col=False, usecols=[2,5,7,8,9,10,11,12,13,14,15,16,17,18])
elon = pd.read_csv('2022_elon_hitters.csv', index_col=False, usecols=[2,5,7,8,9,10,11,12,13,14,15,16,17,18])
hof = pd.read_csv('2022_hof_hitters.csv', index_col=False, usecols=[2,5,7,8,9,10,11,12,13,14,15,16,17,18])
mon = pd.read_csv('2022_mon_hit.csv', index_col=False, usecols=[2,5,7,8,9,10,11,12,13,14,15,16,17,18])
sb = pd.read_csv('2022_sb_hitters.csv', index_col=False, usecols=[2,5,7,8,9,10,11,12,13,14,15,16,17,18])
tow = pd.read_csv('2022_tow_hit.csv', index_col=False, usecols=[2,5,7,8,9,10,11,12,13,14,15,16,17,18])
jmu = pd.read_csv('2022_jmu_hitters.csv', index_col=False, usecols=[2,5,7,8,9,10,11,12,13,14,15,16,17,18])
ncat = pd.read_csv('2022_ncat_hitters.csv', index_col=False, usecols=[2,5,7,8,9,10,11,12,13,14,15,16,17,18])
sh = pd.read_csv('2022_sh_hitters.csv', index_col=False, usecols=[2,5,7,8,9,10,11,12,13,14,15,16,17,18])
ek = pd.read_csv('2022_ek_hitters.csv', index_col=False, usecols=[2,5,7,8,9,10,11,12,13,14,15,16,17,18])

#remove incomplete coc game
coc = coc.iloc[:6891]

pxp = pd.concat([neu,coc,dele,elon,hof,mon,sb,tow,jmu,ncat,sh,ek], axis=0)


# create a new column to store the game ID
pxp['game_pk'] = None

# create a new column to store the previous inning value
pxp['Previous Inning'] = pxp['Inning'].shift(1)

# initialize a variable to store the current game ID
game_id = 0

pxp = pxp.reset_index()

# create a new column to store the score for the current inning
pxp['bat_score'] = None

# create new columns for each base
pxp['on_1b'] = 0
pxp['on_2b'] = 0
pxp['on_3b'] = 0

# iterate through the rows of the dataframe
for index, row in pxp.iterrows():
    
    if (row['Inning'] in ['T1','B1']) and (row['Previous Inning'] not in ['T1','B1']):
        game_id += 1
    pxp.at[index, 'game_pk'] = game_id
    
    score = row['Score'].split('-')
    if row['Inning'][0] == 'T':
        pxp.at[index, 'bat_score'] = int(score[0])
    else:
        pxp.at[index, 'bat_score'] = int(score[1])
    
    count = row['Count'].split('-')
    pxp.at[index, 'balls'] = count[0]
    pxp.at[index, 'strikes'] = count[1]
    
    if '1B' in row['Runners On Base']:
        pxp.at[index, 'on_1b'] = 1
    if '2B' in row['Runners On Base']:
        pxp.at[index, 'on_2b'] = 1
    if '3B' in row['Runners On Base']:
        pxp.at[index, 'on_3b'] = 1

        
pxp['state'] = (
    pxp['balls'].astype(str) + 
    pxp['strikes'].astype(str) + 
    pxp['Outs'].astype(str) + 
    pxp['on_1b'].astype(str) + 
    pxp['on_2b'].astype(str) + 
    pxp['on_3b'].astype(str)
)

pxp.rename(columns={'Inning':'inning'}, inplace= True)

pxp['event'] = pxp['BIP Result'].fillna(pxp['Pitch Result'])

pxp.drop(columns=['index','Previous Inning'], inplace = True)

pxp.dropna(subset=['game_pk', 'bat_score',
       'on_1b', 'on_2b', 'on_3b', 'balls', 'strikes',
       'state', 'event'], inplace = True)

In [4]:
# Group the data by game, inning, and top/bottom of inning
half_innings = pxp.groupby(['game_pk', 'inning'])

# Create an empty list to store the half-inning dataframes
half_inning_dfs = []

# Iterate over the groups in the half_innings object
for name, group in half_innings:
    # Apply the create_half_inning_df function to the group
    half_inning_df = create_half_inning_df(group)
    # Append the resulting dataframe to the half_inning_dfs list
    half_inning_dfs.append(half_inning_df)

for df in half_inning_dfs:

    df['post_bat_score'] = df['bat_score'].shift(-1).fillna(df['bat_score'])

    # Select the last row of the dataframe
    last_row = df.iloc[-1]

    # Get the value of the 'column_name' column in the last row
    last_row_value = last_row['post_bat_score']

    # Assign the value of 'column_name' in the last row to the entire 'column_name' column
    df['runs_by_end'] = last_row_value - df['bat_score']

# itterate over each pitch and record the runs by end for each state and ammount of times the state occured
for df in half_inning_dfs:
    for index, row in df.iterrows():
        if row['state'] in globals():
            globals().get(row['state'])[0] += row['runs_by_end']
            globals().get(row['state'])[1] += 1
        else:
            globals()[row['state']] = [row['runs_by_end'], 1]

states = set(pxp['state'])

for state in states:
    globals()[state] = globals()[state][0]/globals()[state][1]
    
list_of_states_and_run_expectancy = []
for state in states:
    list_of_states_and_run_expectancy.append((state,globals()[state]))
    
run_expectancy_dict = dict(list_of_states_and_run_expectancy)

pxp['state'] = pxp['state'].map(run_expectancy_dict)

# Use the map() function to apply the map_state_column() function to each dataframe in the list
half_inning_dfs = list(map(lambda x: map_state_column(x, run_expectancy_dict), half_inning_dfs))

# Use the map() function to apply the calc_delta_run_expectancy() function to each dataframe in the list
half_inning_dfs = list(map(calc_delta_run_expectancy, half_inning_dfs))

# Concatenate the dataframes in the list into a single dataframe
pxp_updated = pd.concat(half_inning_dfs)

In [5]:
pxp_updated['result'] = pxp_updated['BIP Type'].fillna(pxp_updated['Pitch Result'])

In [6]:
results = ['ball',
'called_strike',
'fly_ball',
'foul',
'foul_pitchout',
'foul_tip',
'ground_ball',
'hit_by_pitch',
'line_drive',
'popup',
'swinging_strike']

In [7]:
pxp_updated_clean = pxp_updated.loc[pxp_updated['result'].isin(results)]

In [8]:
# create runs_scored column for the total runs scored after an event
pxp_updated['runs_scored'] = pxp_updated['post_bat_score'] - pxp_updated['bat_score']

# fill empty events rows with the description
pxp_updated['events'] = pxp_updated['BIP Result'].fillna(pxp_updated['Pitch Result'])

# add runs scored to delta run expectancy
pxp_updated['delta_run_expectancy'] += pxp_updated['runs_scored']

pxp_updated['delta_run_expectancy'] = pd.to_numeric(pxp_updated['delta_run_expectancy'])

In [11]:
pxp_updated[['events', 'delta_run_expectancy']].groupby('events').mean().sort_values(
    'delta_run_expectancy',ascending = False)

,delta_run_expectancy
events,
Home Run,1.360188
Home Run Inside The Park,1.087299
Triple,1.050868
Ground Rule Double,0.798817
Double,0.767256
Fielder's Choice - Safe,0.746041
Field Error,0.575465
Single,0.538411
BIP,0.266949


In [12]:
event_rv('Single')

,delta_run_expectancy
state,
000000,0.469973
000001,0.436742
000010,0.844524
000011,0.178690
000100,0.740234
...,...
322011,1.500256
322100,0.221707
322101,0.860749
